In [11]:
# import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import classification_report
# from sklearn.tree import DecisionTreeClassifier
# import operator
# from math import log
# from collections import Counter
# from statistics import mean

In [12]:
# calculate gini index of dataset
# cycle through each attribute and do the following
#   split data by attribute value and calculate individual gini indices and take weighted average
# choose the attribute with max information gain to split on
# repeat till? information gain is 0?

In [13]:
mushrooms = pd.read_csv('mushrooms.csv')

# data split
mushrooms_train, mushrooms_rest = train_test_split(mushrooms, test_size=0.3)
mushrooms_test, mushrooms_validate = train_test_split(mushrooms_rest, test_size=0.33)

# mushrooms_train.info()
# mushrooms_test.info()
# mushrooms_validate.info()

In [14]:
# X_test = mushrooms_train.drop(['class'], axis=1)
# y_test = mushrooms_train['class']

# for index, row in X_test.iterrows():
#     for value in row:
#         print(f"{ord(value)-ord('a')+1}, {value}")

# clf = DecisionTreeClassifier()
# # clf.fit(X_test, y_test)

In [15]:
# Function to calculate gini index of data
def impurity(data):
    try:
        count_p = data['class'].value_counts()['p']
    except KeyError:
        count_p = 0
    try:
        count_e = data['class'].value_counts()['e']
    except KeyError:
        count_e = 0
    gini = 1 - (count_p/len(data))**2 - (count_e/len(data))**2
    return gini

# print(impurity(mushrooms_train))

# Get possible values of each attribute into a dict
attribute_values = {}
for column in mushrooms_train.columns:
    if column != 'class':
        attribute_values[column] = mushrooms_train[column].unique()

# print(attribute_values['habitat'])

In [16]:
# function pick the attribute splitting by which resluts in minimum gini index
def attribute_choice(data):
    min_gini = 1
    for column in data.columns:
        if column != 'class':
            gini = 0
            for attribute_value in attribute_values[column]:
                if (len(data[data[column] == attribute_value]) == 0):
                    continue
                gini += impurity(data[data[column] == attribute_value]) * len(data[data[column] == attribute_value]) / len(data)
            if gini < min_gini:
                min_gini = gini
                min_gini_column = column
    return min_gini_column, min_gini

# print(attribute_choice(mushrooms_train[mushrooms_train['odor'] == 'n']))

In [17]:
# print(mushrooms_train[mushrooms_train['odor'] == 'n']['class'].value_counts())

In [18]:
decision_tree = {}

# picks best attribute at each level and recursively builds the tree
def build_tree(data):
    if (impurity(data) == 0):
        return {'class': data['class'].iloc[0]}
    attribute_name = attribute_choice(data)[0]
    return_dict = {'attribute': attribute_name}
    for attribute_value in attribute_values[attribute_name]:
        if (len(data[data[attribute_name] == attribute_value]) == 0):
            continue
        return_dict[attribute_value] = build_tree(data[data[attribute_name] == attribute_value])
    return return_dict

decision_tree = build_tree(mushrooms_train)
# print(decision_tree)

In [19]:
# runs through the tree recursdively till it hits a class
def classify(tree, data):
    if ('class' in tree):
        return tree['class']
    attribute_name = tree['attribute']
    data_value = data[attribute_name]
    return classify(tree[data_value], data)

# iterating over validation set to tune pruning parameters
i = 0
for index, row in mushrooms_validate.iterrows():
    val = classify(decision_tree, row.drop('class'))
    if (row['class'] == val):
        i += 1
        
# print('accuracy:', i, len(mushrooms_validate))

In [20]:
# accuracy check on test set
i = 0
for index, row in mushrooms_test.iterrows():
    val = classify(decision_tree, row.drop('class'))
    if (row['class'] == val):
        i += 1
        
print('accuracy:', i, len(mushrooms_test))

accuracy: 1633 1633
